In [38]:
import openeo
import openeo.processes as eop
import json
from generate import create_ppi_cube

In [39]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [40]:
spatial_extent = {
    "west": 4.22,
    "south": 51.16,
    "east": 4.377856,
    "north": 51.26
}
geom = {
    "type": "Polygon",
    "coordinates": [[
        [spatial_extent["west"], spatial_extent["south"]],
        [spatial_extent["east"], spatial_extent["south"]],
        [spatial_extent["east"], spatial_extent["north"]],
        [spatial_extent["west"], spatial_extent["north"]],
        [spatial_extent["west"], spatial_extent["south"]]
    ]]
}

temporal_extent = ["2024-06-01", "2024-08-30"]
ppi_cube = create_ppi_cube(
    connection=connection,
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    geom=geom
)
ppi_cube.execute_batch(out_format="GTiff", title="ppi_example")



Deriving band listing from unordered `item_assets`


0:00:00 Job 'j-260116154518437fb6805812c6500ae8': send 'start'
0:00:17 Job 'j-260116154518437fb6805812c6500ae8': created (progress 0%)
0:00:22 Job 'j-260116154518437fb6805812c6500ae8': created (progress 0%)
0:00:29 Job 'j-260116154518437fb6805812c6500ae8': created (progress 0%)
0:00:37 Job 'j-260116154518437fb6805812c6500ae8': created (progress 0%)
0:00:47 Job 'j-260116154518437fb6805812c6500ae8': created (progress 0%)
0:00:59 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:01:15 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:01:34 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:01:58 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:02:28 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:03:05 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:03:52 Job 'j-260116154518437fb6805812c6500ae8': running (progress N/A)
0:04:51 Job 'j-260116154518437fb6805812c6500ae8': finished (progre

<BatchJob job_id='j-260116154518437fb6805812c6500ae8'>

In [33]:
from itertools import product
s2 = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    bands=["B04", "B08", "sunZenithAngles"],
)

s2_scl = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    bands=["SCL"],
)

masks = [s2_scl]
scl = s2_scl.band("SCL")
mask = ((scl == 3) | (scl == 8) | (scl == 9) | (scl == 10)) * 1
masks.append(mask.add_dimension(name="bands", label="SCL_initial_mask").save_result(format="GTiff"))

for k1, k2 in product(range(1, 21, 6), range(1, 201, 60)):
    mask = s2_scl.process(
        "to_scl_dilation_mask",
        data = s2_scl,
        kernel1_size = k1,
        kernel2_size = k2
    )*1
    mask = mask.rename_labels(dimension="bands", target=[f"SCL_dilation_{k1}_{k2}"]).save_result(format="GTiff")
    masks.append(mask)

result = openeo.MultiResult(masks)
job = result.create_job(title="scl_dilation_kernels_test")
job.start()



<BatchJob job_id='j-2601161521074c4ab25830ca64743670'>

In [ ]:
from pathlib import Path

result_folder = Path("results")
result_folder.mkdir(exist_ok=True)

job.download_results(result_folder)